In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../assets/capstone-assets/CCRPI/master.csv')

In [3]:
df.shape

(1883, 58)

In [6]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

,year,sysID,sysName,schoolID,schoolName,grd,ccrpi,street,city,county,...,pctNative,pctBlck,pctHisp,pctMulti,pctWhite,pctMigrant,pctSPED,pctESOL,pctAltProg,pctGiftd
0,2015,601,Appling County,103,Appling County High School,H,73.6,482 Blackshear Hwy,Baxley,Appling County,...,0,22,11,2,64,4,12.7,1.3,6.8,6.8
1,2015,601,Appling County,177,Appling County Elementary School,E,68.8,680 Blackshear Hwy,Baxley,Appling County,...,0,29,19,4,47,7,12.8,8.7,0.0,2.0
2,2015,601,Appling County,195,Appling County Middle School,M,68.8,2997 Blackshear Hwy,Baxley,Appling County,...,0,22,12,4,62,4,11.2,1.4,2.2,7.0
3,2015,601,Appling County,1050,Altamaha Elementary School,E,72.3,344 Altamaha School Rd,Baxley,Appling County,...,0,6,6,2,86,2,13.2,4.1,0.0,6.2
4,2015,601,Appling County,5050,Fourth District Elementary School,E,68.2,13396 Blackshear Hwy,Surrency,Appling County,...,0,9,9,5,78,5,8.6,2.6,0.0,2.0


In [7]:
df.describe()

/Users/michaelmainzer/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3403: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


,year,sysID,schoolID,ccrpi,zip,locClass,totStudnts,crprlPun,dtntn,iss,...,pctNative,pctBlck,pctHisp,pctMulti,pctWhite,pctMigrant,pctSPED,pctESOL,pctAltProg,pctGiftd
count,1883.0,1883.000000,1883.000000,1883.000000,1883.000000,1881.000000,1883.000000,1883.000000,1883.000000,1883.000000,...,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1883.000000,1783.000000
mean,2015.0,677.570898,1062.330324,72.589166,30857.360064,27.700159,836.787573,5.166755,5.702071,162.216144,...,0.106213,36.849708,12.725438,3.292087,44.120552,0.296867,11.152416,4.928147,1.635582,10.421088
std,0.0,48.310595,1480.929202,12.006565,1472.510844,11.262886,478.702682,23.279748,45.027680,269.171483,...,0.328230,29.430681,14.271004,1.842259,28.863890,1.214299,3.735857,8.790000,9.122699,7.809462
min,2015.0,601.000000,100.000000,29.800000,30004.000000,11.000000,38.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000
25%,2015.0,634.000000,184.000000,64.300000,30157.000000,NaN,537.000000,0.000000,0.000000,0.000000,...,0.000000,12.000000,4.000000,2.000000,16.000000,0.000000,8.900000,0.200000,0.000000,NaN
50%,2015.0,667.000000,279.000000,73.000000,30507.000000,NaN,729.000000,0.000000,0.000000,31.000000,...,0.000000,30.000000,8.000000,3.000000,47.000000,0.000000,11.000000,1.600000,0.000000,NaN
75%,2015.0,713.000000,1105.000000,81.350000,31087.000000,NaN,983.000000,0.000000,0.000000,226.000000,...,0.000000,57.000000,15.000000,4.000000,69.000000,0.000000,13.150000,5.400000,0.700000,NaN
max,2015.0,793.000000,5567.000000,108.000000,39897.000000,43.000000,4042.000000,437.000000,1069.000000,2355.000000,...,4.000000,99.000000,96.000000,11.000000,98.000000,19.000000,65.900000,78.000000,100.000000,63.500000
